In [25]:
!pip install eli5

# New Section

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [27]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/shoes_prices.csv', low_memory=False)
df_usd = df[df.prices_currency == 'USD'].copy()
df_usd['prices_amountmin'] = df_usd.prices_amountmin.astype(np.float)
filter_max = np.percentile(df_usd['prices_amountmin'],99)
df_usd_filter = df_usd[df_usd['prices_amountmin']<filter_max]

In [29]:
df = df_usd_filter
df.shape

(17678, 48)

In [0]:
df.to_csv('data/shoes_prices.csv', index=False)
df = pd.read_csv('data/shoes_prices.csv', low_memory=False)

In [0]:
def run_model (feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [32]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-46.11708402823731, 1.000601732312841)

In [33]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-45.777804709663116, 1.0817565884953784)

In [34]:
df.features

0        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
1        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
2        [{"key":"Gender","value":["Men"]},{"key":"Colo...
3        [{"key":"Gender","value":["Men"]},{"key":"Colo...
4        [{"key":"Gender","value":["Men"]},{"key":"Colo...
                               ...                        
17673    [{"key":"Style","value":["Desert Boots"]},{"ke...
17674    [{"key":"Shoe Size","value":["9.5"]},{"key":"G...
17675    [{"key":"Gender","value":["Unisex"]},{"key":"F...
17676    [{"key":"Gender","value":["Unisex"]},{"key":"F...
17677    [{"key":"Gender","value":["Unisex"]},{"key":"F...
Name: features, Length: 17678, dtype: object

Start here

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item ['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df ['features_parsed'] = df['features'].map(parse_features)

In [36]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [37]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

469

In [38]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan )

In [0]:
keys_stat= {}
for key in keys:
  keys_stat[key] = df[ False  == df[get_name_feat(key)].isnull()].shape[0]/df.shape[0] * 100

In [40]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 49.60968435343365,
 'color': 48.71026134178074,
 'gender': 51.074782215182715,
 'manufacturer part number': 36.966851453784365,
 'material': 35.52438058603914}

In [0]:
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_gender_cat'] = df['feat_color'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [42]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand].shape

(8728, 524)

In [43]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-45.81943705978364, 1.0610947524062577)

In [54]:
feats_cat = [x for  x in df.columns if 'cat' in x]
feats_cat


['categories',
 'brand_cat',
 'feat_location - country',
 'feat_shoe category',
 'feat_recommended location',
 'feat_fabrication',
 'feat_certifications and listings',
 'feat_clothing category',
 'feat_catalog',
 'feat_location - city/state',
 'feat_multi pack indicator',
 'feat_color_cat',
 'feat_brand_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_frame color_cat',
 'feat_contained battery type_cat',
 'feat_case material_cat',
 'feat_color mapping_cat',
 'feat_flame resistant_cat',
 'feat_eye protection type_cat',
 'feat_heel height_cat',
 'feat_looks best on these face shapes:_cat',
 'feat_style:_cat',
 'feat_removable liner_cat',
 'feat_features & fastening_cat',
 'feat_number of compartments_cat',
 'feat_sku#_cat',
 'feat_variant group id_cat',
 'feat_measurements:_cat',
 'feat_country/ organization_cat',
 'feat_hardsided or softsided_cat',
 'feat_assembled in country of origin_cat',
 'feat_case ty

In [57]:
feats = ['brand_cat', 'feat_metal type_cat','feat_brand_cat', 'feat_shape_cat' ,'feat_gender_cat',  'feat_material_cat', 'feat_sport_cat', 'feat_style_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats,model)

(-45.21090962140022, 1.5317240956008213)

In [58]:
X = df[feats].values
y  = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names = feats)

Weight,Feature
0.1254 ± 0.0050,feat_material_cat
0.0565 ± 0.0038,feat_brand_cat
0.0478 ± 0.0013,brand_cat
0.0338 ± 0.0024,feat_gender_cat
0.0255 ± 0.0020,feat_style_cat
0.0123 ± 0.0018,feat_shape_cat
0.0012 ± 0.0001,feat_sport_cat
0.0000 ± 0.0000,feat_metal type_cat


In [52]:
df['brand'].value_counts(normalize = True)

nike            0.099502
puma            0.034450
vans            0.021835
new balance     0.020930
ralph lauren    0.016009
                  ...   
jockey          0.000057
cabela's        0.000057
a. testoni      0.000057
im survival     0.000057
milwaukee       0.000057
Name: brand, Length: 1704, dtype: float64